In [1]:
from sko.PSO import PSO
import pandas as pd
import numpy as np

In [21]:
#run no-veh_id : [pair_id] 
following_CAV_dict={'8-206': [643], '8-209': [645], '1-9': [6], '1-16': [7], '1-68': [8], '1-77': [9], '8-180': [647], 
                    '8-150': [649], '7-21': [424], '7-69': [426], '7-44': [427], '7-78': [428], '7-84': [429], 
                    '7-49': [430], '21-71': [2172], '21-3': [2173], '19-29': [1859], '18-11': [1667], '19-51': [1860], 
                    '19-55': [1861], '19-48': [1862], '19-78': [1863], '18-17': [1668], '20-2': [2049], '20-4': [2050], 
                    '18-4': [1669], '20-34': [2051], '20-30': [2052], '11-32': [1495], '11-36': [1496], '11-105': [1497], 
                    '11-30': [1498], '3-53': [219], '3-31': [220], '3-35': [222], '3-68': [223], '3-71': [224], '3-83': [225],
                    '20-49': [2053], '9-64': [1393], '21-17': [2168], '21-39': [2169], '21-48': [2170], '21-88': [2171], 
                    '8-6': [636], '8-60': [638], '8-61': [637], '7-41': [425], '18-32': [1670]}
following_HV_dict={'8-206': [1091, 1104], '8-209': [1097, 1151], '1-9': [11, 63, 82, 111], '1-16': [18], '1-68': [32, 100], '1-77': [58, 134, 136], '8-180': [1089, 1099, 1141, 1143], '8-150': [615, 1174, 667, 940, 973], '7-21': [417, 433, 437, 477], '7-69': [528, 538, 480, 492, 506], '7-44': [529, 533, 563, 475, 487, 508], '7-78': [568, 482], '7-84': [545, 559, 564], '7-49': [406, 440], '21-71': [2143, 2234, 2293], '21-3': [2218], '19-29': [1874], '18-11': [1651], '19-51': [1916, 1939, 1979], '19-55': [1918, 1943, 1949, 2023], '19-48': [1886, 1912, 1926, 1953, 1968], '19-78': [1858, 1936, 1967, 1998, 2003], '18-17': [1634, 1648, 1660, 1696, 1728, 1736], '20-2': [2044, 2040], '20-4': [2035], '18-4': [1656, 1662, 1671, 1719], '20-34': [2102], '20-30': [2082, 2103, 2030], '11-32': [1501], '11-36': [1502], '11-105': [1612, 1490], '11-30': [1590, 1479, 1493, 1504], '3-53': [198, 208, 261, 339], '3-31': [187, 207], '3-35': [199, 206, 266], '3-68': [195, 232, 286, 357], '3-71': [218, 288, 297, 307, 356], '3-83': [317, 325, 353], '20-49': [2046], '9-64': [1412, 1439], '21-17': [2148, 2184], '21-39': [2165], '21-48': [2155], '21-88': [2142, 2235, 2257, 2274], '8-6': [658], '8-60': [789, 805, 1117, 1144, 1160], '8-61': [652, 790], '7-41': [436, 450, 500], '18-32': [1632, 1691, 1715]}


In [3]:
#get the run_id list, veh_id_list, and pair_id_list
CAV_judge=True
run_id_list=[]
veh_id_list=[]
pair_id_list=[]
if CAV_judge:
    following_veh_dict=following_CAV_dict
else:
    following_veh_dict=following_HV_dict
for item in list(following_veh_dict.keys()):
    run_no=item.split("-")[0]
    veh_id=item.split("-")[1]
    run_id_list.append(run_no)
    veh_id_list.append(veh_id)
    pair_id_list.append(following_veh_dict[item])
    

In [83]:
#update vehicle state
def veh_state_update(xt,vt,at,time_step):
    vt_next=vt+at*time_step
    xt_next=xt+vt*time_step+0.5*at*time_step*time_step
    return xt_next,vt_next

In [84]:
#define the objective function of PSO
def objective_function(Vars):
        x1 = Vars[0]
        x2 = Vars[1]
        x3 = Vars[2]
        x4 = Vars[3]
        x5 = Vars[4]     
        total_loss=0
        for current_tra_i in range(len(x_leader_current_list)):
            #traffic state of the leading vehicle at current time step
            x_leader_current=x_leader_current_list[current_tra_i]
            v_leader_current=v_leader_current_list[current_tra_i]
            a_leader_current=a_leader_current_list[current_tra_i]
            
            #traffic state of the following vehicle at current time step
            x_follower_current=x_follower_current_list[current_tra_i]
            v_follower_current=v_follower_current_list[current_tra_i]
            a_follower_current=a_follower_current_list[current_tra_i]
            
            #traffic state of the leading vehicle at next time step
            x_leader_next=x_leader_next_list[current_tra_i]
            v_leader_next=v_leader_next_list[current_tra_i]
            a_leader_next=a_leader_next_list[current_tra_i]
            
            #traffic state of the following vehicle at next time step
            x_follower_next=x_follower_next_list[current_tra_i]
            v_follower_next=v_follower_next_list[current_tra_i]
            a_follower_next=a_follower_next_list[current_tra_i]
            
            spacing=spacing_list[current_tra_i]
            delta_v=delta_v_list[current_tra_i]
            vt_follower=v_follower_current[0]
            ds=spacing[0]
            dv=delta_v[0]
            xt_follower=x_follower_current[0]
            current_traj_loss=0
            for i in range(len(v_follower_current)):
                sn = x2 + vt_follower * x3 + vt_follower * dv / (2 * np.sqrt(x4 * x5))
                a_follower = x4 * (1 - (vt_follower / x1) ** 4 - (sn / ds) ** 2)
                xt_follower_next,vt_follower_next=veh_state_update(xt_follower,vt_follower,a_follower,0.1)
                vt_follower=vt_follower_next
                xt_follower=xt_follower_next
                vt_leader_next=v_leader_next[i]
                xt_leader_next=x_leader_next[i]
                dv=vt_follower-vt_leader_next
                ds=xt_leader_next-xt_follower_next-5
                xt_follower_next_true=x_follower_next[i]
                vt_follower_next_true=v_follower_next[i]
                current_traj_loss+=np.abs(np.subtract(xt_follower, xt_follower_next_true))
            total_loss+=current_traj_loss
        return total_loss


In [86]:
parameter_dict={}
#loop
for i in range(len(run_id_list)):
    run_no=run_id_list[i]
    veh_id=veh_id_list[i]
    pair_id=pair_id_list[i]
    key=run_no+'-'+veh_id+'-'+str(pair_id)
    print("run_no:",run_no,"veh_id:",veh_id,"pair_id:",pair_id)
    x_leader_current_list=[]
    v_leader_current_list=[]
    a_leader_current_list=[]
    x_follower_current_list=[]
    v_follower_current_list=[]
    a_follower_current_list=[]

    x_leader_next_list=[]
    v_leader_next_list=[]
    a_leader_next_list=[]
    x_follower_next_list=[]
    v_follower_next_list=[]
    a_follower_next_list=[]
    spacing_list=[]
    delta_v_list=[]
    timestamp_list=[]
    for current_pair_id in pair_id:
        file_name='run-'+run_no+'-veh-'+veh_id+'-pair-'+str(current_pair_id)+'.csv'
        if CAV_judge:
            data_path="../Data/CAV/"
        else:
            data_path="../Data/HV/"
        #read trajectory data
        veh_data=pd.read_csv(data_path+file_name)
        x_leader_current=veh_data['xLeaderReal'].tolist()
        v_leader_current=veh_data['vLeaderReal'].tolist()
        a_leader_current=veh_data['aLeaderReal'].tolist()

        x_follower_current=veh_data['xFollowerReal'].tolist()
        v_follower_current=veh_data['vFollowerReal'].tolist()
        a_follower_current=veh_data['aFollowerReal'].tolist()

        x_leader_next=veh_data['xLeaderReal_next'].tolist()
        v_leader_next=veh_data['vLeaderReal_next'].tolist()
        a_leader_next=veh_data['aLeaderReal_next'].tolist()

        x_follower_next=veh_data['xFollowerReal_next'].tolist()
        v_follower_next=veh_data['vFollowerReal_next'].tolist()
        a_follower_next=veh_data['aFollowerReal_next'].tolist()

        spacing=veh_data['spacing'].tolist()
        delta_v=veh_data['speed_difference'].tolist()
        timestamp=veh_data['timestamp'].tolist()
       
        x_leader_current_list.append(x_leader_current)
        v_leader_current_list.append(v_leader_current)
        a_leader_current_list.append(a_leader_current)
        x_follower_current_list.append(x_follower_current)
        v_follower_current_list.append(v_follower_current)
        a_follower_current_list.append(a_follower_current)

        x_leader_next_list.append(x_leader_next)
        v_leader_next_list.append(v_leader_next)
        a_leader_next_list.append(a_leader_next)
        x_follower_next_list.append(x_follower_next)
        v_follower_next_list.append(v_follower_next)
        a_follower_next_list.append(a_follower_next)
        spacing_list.append(spacing)
        delta_v_list.append(delta_v)
        timestamp_list.append(timestamp)
    #Apply PSO     
    pso = PSO(func=objective_function, n_dim=5, pop=40, max_iter=150, lb=[10,1,0.6,0.28,0.47], ub=[35,5,6,3.41,3.41], w=0.8, c1=0.5, c2=0.5)
    pso.run()
    parameter_dict[key]=pso.gbest_x
    print('best_x is ', pso.gbest_x, 'best_y is', pso.gbest_y)

run_no: 8 veh_id: 206 pair_id: [1091, 1104]
best_x is  [35.          5.          1.12208102  0.36896234  0.64708064] best_y is [494.3524583]
run_no: 8 veh_id: 209 pair_id: [1097, 1151]


C:\Users\wangq2\AppData\Local\Temp\ipykernel_23348\3625080482.py:36: RuntimeWarning: overflow encountered in double_scalars
  sn = x2 + vt_follower * x3 + vt_follower * dv / (2 * np.sqrt(x4 * x5))
C:\Users\wangq2\AppData\Local\Temp\ipykernel_23348\3625080482.py:39: RuntimeWarning: overflow encountered in double_scalars
  a_follower = x4 * (1 - (vt_follower / x1) ** 4 - (sn / ds) ** 2)
C:\Users\wangq2\AppData\Local\Temp\ipykernel_23348\3625080482.py:36: RuntimeWarning: invalid value encountered in double_scalars
  sn = x2 + vt_follower * x3 + vt_follower * dv / (2 * np.sqrt(x4 * x5))


best_x is  [35.          1.97750109  0.6         0.81843202  3.41      ] best_y is [581.62996438]
run_no: 1 veh_id: 9 pair_id: [11, 63, 82, 111]


C:\Users\wangq2\AppData\Local\Temp\ipykernel_23348\3625080482.py:36: RuntimeWarning: overflow encountered in double_scalars
  sn = x2 + vt_follower * x3 + vt_follower * dv / (2 * np.sqrt(x4 * x5))
C:\Users\wangq2\AppData\Local\Temp\ipykernel_23348\3625080482.py:39: RuntimeWarning: overflow encountered in double_scalars
  a_follower = x4 * (1 - (vt_follower / x1) ** 4 - (sn / ds) ** 2)
C:\Users\wangq2\AppData\Local\Temp\ipykernel_23348\3625080482.py:36: RuntimeWarning: invalid value encountered in double_scalars
  sn = x2 + vt_follower * x3 + vt_follower * dv / (2 * np.sqrt(x4 * x5))


best_x is  [35.    1.    0.6   3.41  3.41] best_y is [626.36613233]
run_no: 1 veh_id: 16 pair_id: [18]
best_x is  [35.          1.          0.6         3.41        0.52699764] best_y is [6.52774787]
run_no: 1 veh_id: 68 pair_id: [32, 100]
best_x is  [32.13969724  1.          0.6         3.41        2.30456924] best_y is [760.6945061]
run_no: 1 veh_id: 77 pair_id: [58, 134, 136]
best_x is  [34.08122289  5.          1.08923032  3.41        3.41      ] best_y is [453.00254613]
run_no: 8 veh_id: 180 pair_id: [1089, 1099, 1141, 1143]
best_x is  [24.52928315  5.          0.89718562  0.60279393  1.05458208] best_y is [147.39516607]
run_no: 8 veh_id: 150 pair_id: [615, 1174, 667, 940, 973]
best_x is  [15.34185517  1.          2.21790384  0.28        2.14226962] best_y is [133.39929133]
run_no: 7 veh_id: 21 pair_id: [417, 433, 437, 477]
best_x is  [29.93629405  1.          0.6         2.55511063  0.47      ] best_y is [384.59012031]
run_no: 7 veh_id: 69 pair_id: [528, 538, 480, 492, 506]


...

best_x is  [35.          1.          0.6         2.86330554  1.71232517] best_y is [261.78507373]
run_no: 20 veh_id: 49 pair_id: [2046]
best_x is  [35.          4.09995717  0.6         3.41        0.47      ] best_y is [27.30463951]
run_no: 9 veh_id: 64 pair_id: [1412, 1439]
best_x is  [35.         1.         0.6        0.28       2.4530902] best_y is [942.42834883]
run_no: 21 veh_id: 17 pair_id: [2148, 2184]
best_x is  [33.99075593  5.          0.78654537  3.41        1.70272347] best_y is [653.34172135]
run_no: 21 veh_id: 39 pair_id: [2165]
best_x is  [35.          5.          1.44088049  0.49435147  3.40991912] best_y is [65.25188743]
run_no: 21 veh_id: 48 pair_id: [2155]


C:\Users\wangq2\AppData\Local\Temp\ipykernel_23348\3625080482.py:36: RuntimeWarning: overflow encountered in double_scalars
  sn = x2 + vt_follower * x3 + vt_follower * dv / (2 * np.sqrt(x4 * x5))
C:\Users\wangq2\AppData\Local\Temp\ipykernel_23348\3625080482.py:39: RuntimeWarning: overflow encountered in double_scalars
  a_follower = x4 * (1 - (vt_follower / x1) ** 4 - (sn / ds) ** 2)
C:\Users\wangq2\AppData\Local\Temp\ipykernel_23348\3625080482.py:36: RuntimeWarning: invalid value encountered in double_scalars
  sn = x2 + vt_follower * x3 + vt_follower * dv / (2 * np.sqrt(x4 * x5))


best_x is  [29.85486869  5.          1.02408749  2.19708976  0.47      ] best_y is [609.19453371]
run_no: 21 veh_id: 88 pair_id: [2142, 2235, 2257, 2274]


C:\Users\wangq2\AppData\Local\Temp\ipykernel_23348\3625080482.py:36: RuntimeWarning: overflow encountered in double_scalars
  sn = x2 + vt_follower * x3 + vt_follower * dv / (2 * np.sqrt(x4 * x5))
C:\Users\wangq2\AppData\Local\Temp\ipykernel_23348\3625080482.py:39: RuntimeWarning: overflow encountered in double_scalars
  a_follower = x4 * (1 - (vt_follower / x1) ** 4 - (sn / ds) ** 2)
C:\Users\wangq2\AppData\Local\Temp\ipykernel_23348\3625080482.py:36: RuntimeWarning: invalid value encountered in double_scalars
  sn = x2 + vt_follower * x3 + vt_follower * dv / (2 * np.sqrt(x4 * x5))


best_x is  [35.          2.42740327  0.6         3.41        3.41      ] best_y is [545.58623018]
run_no: 8 veh_id: 6 pair_id: [658]
best_x is  [33.7151437   1.42475814  0.6         0.28120964  2.57936166] best_y is [0.52966447]
run_no: 8 veh_id: 60 pair_id: [789, 805, 1117, 1144, 1160]
best_x is  [35.          1.          1.64186519  0.68830125  3.41      ] best_y is [1165.10121086]
run_no: 8 veh_id: 61 pair_id: [652, 790]
best_x is  [24.46238924  5.          2.24599795  0.28        1.85441769] best_y is [55.57389525]
run_no: 7 veh_id: 41 pair_id: [436, 450, 500]
best_x is  [28.95052761  4.99995367  0.60057342  2.18732447  3.41      ] best_y is [944.34277447]
run_no: 18 veh_id: 32 pair_id: [1632, 1691, 1715]
best_x is  [28.69111924  1.          0.6         0.77389201  3.41      ] best_y is [305.52248949]


In [ ]:
#save parameter
parameter_df=pd.DataFrame(parameter_dict)
parameter_df_T=parameter_df.T
parameter_df_T=parameter_df_T.reset_index()
parameter_df_T[['run_no','veh_id','pair_list']]=parameter_df_T.iloc[:,0].str.split('-',expand=True)
parameter_df_T['pair_list']=parameter_df_T['pair_list'].apply(lambda x: [int(num) for num in x.strip('[]').split(',')])
parameter_df_T=parameter_df_T.explode('pair_list')
parameter_df_T.rename(columns={'pair_list':'pair_id'},inplace=True)
parameter_df_T.rename(columns={0:'Free-flow speed',1:'Minimum space',2:'Safe time headway',3:'Maximum acceleration',4:'Confortable deceleration'},inplace=True)
parameter_df_T=parameter_df_T[['run_no','veh_id','pair_id','Free-flow speed','Minimum space','Safe time headway','Maximum acceleration','follower speed']]
if CAV_judge:
    parameter_df_T.to_csv("../parameters/parameters_following_CAV.csv",index=False)
else:
    parameter_df_T.to_csv("../parameters/parameters_following_HV.csv",index=False)